In [12]:
# Assigment 1: Comparative Study of Classification Algorithms.
print("Hola mundo")

Hola mundo


In [13]:
print("Mundo dice ciao")
print("Ciao dice mundo")
print("Mundo dice que ciao lava piati")

Mundo dice ciao
Ciao dice mundo
Mundo dice que ciao lava piati


In [14]:
print("Mundo gioca calcio")

Mundo gioca calcio


In [15]:
print("Giusseppe se puso la peluca")

Giusseppe se puso la peluca


In [16]:
import kagglehub
import os
import shutil

# Dataset directory
dataset_dir = "data/raw"

# Download dataset
path = kagglehub.dataset_download("jiayuanchengala/aid-scene-classification-datasets")
src_dir = os.path.join(path, "AID")

# Move only if not already moved
if not os.path.exists(dataset_dir):
    print("Copying dataset to /content/data/raw...")
    # Note that if I move it then kagglehub wont re-download so we can=t keep consistency
    shutil.copytree(src_dir, dataset_dir)
else:
    print("Dataset already exists at /content/data/raw")

print("✅ Dataset ready at:", dataset_dir)


ModuleNotFoundError: No module named 'kagglehub'

**Manual Data Separation**

In [ ]:
import os
image_paths =  []
labels = []
source_dir = 'data/raw'
categories = [d for d in os.listdir(source_dir)
              if os.path.isdir(os.path.join(source_dir, d))]
for category in categories:
    category_path = os.path.join(source_dir, category)
    files = [os.path.join(category_path, f) for f in os.listdir(category_path)
             if os.path.isfile(os.path.join(category_path, f))]
    image_paths.extend(files)
    labels.extend([category] * len(files))

print(f"Found {len(image_paths)} images in {len(categories)} categories")

Found 10000 images in 30 categories


In [ ]:
import random
from pathlib import Path

# Set random seed for reproducibility
random.seed(42)

# Source directory containing all the category folders
source_dir = 'data/raw'

# Create train and test directories
train_dir = 'data/train'
test_dir = 'data/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for category in categories:
    category_path = os.path.join(source_dir, category)

    if not os.path.exists(category_path):
        print(f"Warning: {category} folder not found, skipping...")
        continue

    # Get all files in the category folder
    files = [f for f in os.listdir(category_path)
             if os.path.isfile(os.path.join(category_path, f))]

    if len(files) == 0:
        print(f"Warning: {category} folder is empty, skipping...")
        continue

    # Shuffle files randomly
    random.shuffle(files)

    # Calculate split point (90% for train)
    split_idx = int(len(files) * 0.9)
    train_files = files[:split_idx]
    test_files = files[split_idx:]

    # Create category subfolders in train and test
    train_category_dir = os.path.join(train_dir, category)
    test_category_dir = os.path.join(test_dir, category)
    os.makedirs(train_category_dir, exist_ok=True)
    os.makedirs(test_category_dir, exist_ok=True)

    # Move files to train
    for file in train_files:
        src = os.path.join(category_path, file)
        dst = os.path.join(train_category_dir, file)
        shutil.copy2(src, dst)

    # Move files to test
    for file in test_files:
        src = os.path.join(category_path, file)
        dst = os.path.join(test_category_dir, file)
        shutil.copy2(src, dst)

    print(f"{category}: {len(train_files)} files to train, {len(test_files)} files to test")

print("\nSplit complete!")

In [ ]:
from collections import defaultdict

# This is for cross-validation (Pytorch and Tensorflow need manual separation)
# To scikit-learning you onl need to pass the training data and it does the division

# Configuration
k_folds = 5
output_base = 'kfolds'
os.makedirs(output_base, exist_ok=True)

print(f"Creating {k_folds}-fold cross-validation split...")

# Collect all files by category
category_files = {}
for category in categories:
    category_path = os.path.join(source_dir, category)
    files = [f for f in os.listdir(category_path)
             if os.path.isfile(os.path.join(category_path, f))]
    random.shuffle(files)  # Shuffle within each category
    category_files[category] = files

# Create fold assignments for each category
fold_assignments = defaultdict(lambda: defaultdict(list))

for category, files in category_files.items():
    n_files = len(files)
    fold_size = n_files // k_folds

    # Assign files to folds ensuring balanced distribution
    for fold_idx in range(k_folds):
        start_idx = fold_idx * fold_size
        # Last fold gets any remaining files
        end_idx = start_idx + fold_size if fold_idx < k_folds - 1 else n_files
        fold_assignments[fold_idx][category] = files[start_idx:end_idx]

# Create fold directories with class subdirectories only
for fold_idx in range(k_folds):
    fold_name = f'fold_{fold_idx + 1}'
    fold_dir = os.path.join(output_base, fold_name)

    # Create class subdirectories in each fold
    for category in categories:
        category_fold_dir = os.path.join(fold_dir, category)
        os.makedirs(category_fold_dir, exist_ok=True)

        # Copy the files assigned to this fold for this category
        files_for_this_fold = fold_assignments[fold_idx][category]

        for file in files_for_this_fold:
            src = os.path.join(source_dir, category, file)
            dst = os.path.join(category_fold_dir, file)
            shutil.copy2(src, dst)

    print(f"\n{fold_name}:")
    print("=" * 50)

    total_files = 0
    for category in categories:
        files_in_fold = fold_assignments[fold_idx][category]
        total_files += len(files_in_fold)
        print(f"  {category}: {len(files_in_fold)} files")

    print(f"  Total in fold: {total_files} files")

print(f"\n\nK-fold split complete!")
print(f"Output directory: {output_base}/")
print(f"Structure: fold_1/, fold_2/, ..., fold_{k_folds}/")
print(f"Each fold contains: {', '.join(categories)} subdirectories with their respective files")

print(f"\n\nK-fold split complete!")
print(f"Output directory: {output_base}/")
print(f"Structure: fold_1/, fold_2/, ..., fold_{k_folds}/")

# **Training**

In [ ]:
# Since scikit-learning doesn't have features extraction for images
# We have to use also openCV to create our dictionary of features
from sklearn.model_selection import train_test_split
import cv2 as cv
import numpy as np
from sklearn.decomposition import PCA
from pathlib import Path



X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=0, stratify=labels)

# We get all descriptors for training set before kfold, does not count as data leakage
# It just avoids calculating them k times
orb = cv.ORB_create() # Fast but poor performance
# sift = cv.SIFT_create(nfeatures=2000) #13gb is not enough for sift(40+ min)
sift = cv.SIFT_create()
# surf = cv.xfeatures2d.SURF_create() Non-free
akaze = cv.AKAZE_create(descriptor_type=cv.AKAZE_DESCRIPTOR_MLDB,
                         descriptor_size=0,   # 486 bits
                         threshold=0.001) #Slightly faster same results
fast = cv.FastFeatureDetector_create()

#Sift with 500 features performs a faster than akaze and just has 2% less accuracy (10 min training and faster kmeans)
# 2000 features takes 8gb rams and takes around 35-40min, but can't do kmeans)
# 1500 features takes 6.7 and around 35 min, but still isnt enough for kmeans
# 1000 features takes 5gb ram and around 31 min, but still doesnt work with kmeans
# Kmeans consumes around 6gb ram, so any detector method that keeps more than 6gb too will probably crash

# Note that this also affect the kmeans
# Seems like more computing time on getting labels is also translated to longer kmean

image_descriptors = []
pca = PCA(n_components=64) # Reduce descriptor dimension by half from 128 to 64
output_dir = "descriptors"
os.makedirs(output_dir, exist_ok=True)

for idx, x in enumerate(X_train):
  # print(x, type(x))
  p = Path(x)
  # print(p.stem, type(p.stem))
  img = cv.imread(x, cv.IMREAD_GRAYSCALE)
  keypoints = sift.detect(img, None)
  keypoints, descriptors = sift.compute(img, keypoints)
  # Reducing precision from float32 to float16
  # if descriptors is not None:
    # descriptors = descriptors.astype('float16')
    # descriptors = pca.fit_transform(descriptors)
  np.save(f"{output_dir}/{p.stem}.npy", descriptors)
  image_descriptors.append(descriptors)





Corrupt JPEG data: premature end of data segment


In [ ]:
def load_desc_safe(file):
    path = os.path.join(desc_folder, file)
    desc = np.load(path, allow_pickle=True)

    if desc is None or desc.size == 0:
        return None

    # guarantee correct shape
    desc = np.asarray(desc, dtype=np.float32)

    if desc.ndim != 2 or desc.shape[1] != 128:
        return None

    return desc

# desc_dir = "descriptors"

# for fname in os.listdir(desc_dir):
#     path = os.path.join(desc_dir, fname)

#     try:
#         desc = np.load(path, allow_pickle=True)
#     except:
#         print("Unreadable file:", fname, "— replacing with empty.")
#         desc = None
#         np.save(path, desc)
#         continue

#     # Case 0: Scalar (e.g., nan)
#     if desc.ndim == 0:
#         print("Scalar descriptor:", fname, "— fixing to empty.")
#         desc = None
#         np.save(path, desc)
#         continue

#     # Case 1: None or empty
#     if desc is None or desc.size == 0:
#         print("Fixing empty:", fname)
#         desc = None
#         np.save(path, desc)
#         continue

#     # Case 2: 1D vector
#     if desc.ndim == 1:
#         if desc.shape[0] == 128:  # single descriptor
#             print("Fixing single vector:", fname)
#             desc = desc.reshape(1,128)
#         else:
#             print("Invalid 1D descriptor:", fname, "— fixing to empty.")
#             desc = None

#         np.save(path, desc)
#         continue

#     # Case 3: 2D but wrong feature length
#     if desc.ndim == 2 and desc.shape[1] != 128:
#         print("Invalid 2D shape:", fname, desc.shape, "— fixing to empty.")
#         desc = None
#         np.save(path, desc)
#         continue




In [ ]:
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import normalize
from abc import ABC, abstractmethod
from typing import List, Optional, Union
from pathlib import Path


def descriptor_batch_generator(data_loader, load_func=None, max_descriptors=10000):
    """
    Yields batches of descriptors with a total number of rows up to max_descriptors.

    Args:
        data_loader: iterable of items to load descriptors from.
        load_func: optional function(item) -> descriptor array
        max_descriptors: max total rows per batch

    Yields:
        np.ndarray of shape (~max_descriptors, n_features)
    """
    batch = []
    current_size = 0   # number of descriptors accumulated

    for item in data_loader:
        data = load_func(item) if load_func else item

        # Skip empty or None
        if data is None or data.size == 0:
            continue

        # Ensure 2D
        if data.ndim == 1:
            data = data.reshape(1, -1)

        batch.append(data)
        current_size += data.shape[0]

        # If batch is full, yield it
        if current_size >= max_descriptors:
            yield np.vstack(batch)
            batch = []
            current_size = 0

    # Yield remainder
    if batch:
        yield np.vstack(batch)


# ============================================================================
# Base Classes
# ============================================================================

class ClusteringAlgorithm(ABC):
    """Base class for clustering algorithms."""

    def __init__(self, n_clusters: int, random_state: int = 42):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.model = None
        self._is_fitted = False

    @abstractmethod
    def fit(self, X: np.ndarray):
        """Fit the clustering model."""
        pass

    @abstractmethod
    def fit_iterative(self, data_loader, load_func: Optional[callable] = None):
        """Fit iteratively for large datasets."""
        pass

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict cluster assignments."""
        if not self._is_fitted:
            raise ValueError("Model must be fitted before predicting.")
        return self.model.predict(X)

    @property
    def cluster_centers_(self) -> np.ndarray:
        """Get cluster centers."""
        if not self._is_fitted:
            raise ValueError("Model must be fitted first.")
        return self.model.cluster_centers_


class EncodingAlgorithm(ABC):
    """Base class for encoding algorithms (BoVW, VLAD, Fisher)."""

    def __init__(self, clustering: ClusteringAlgorithm):
        self.clustering = clustering

    @abstractmethod
    def encode(self, descriptors: np.ndarray) -> np.ndarray:
        """Encode descriptors into a fixed-length vector."""
        pass

    @abstractmethod
    def encode_batch(self, descriptors_list: List[np.ndarray]) -> np.ndarray:
        """Encode a batch of descriptor sets."""
        pass

# ============================================================================
# Clustering Implementations
# ============================================================================

class KMeansClustering(ClusteringAlgorithm):
    """Standard KMeans clustering."""

    def __init__(self, n_clusters: int, random_state: int = 0, **kwargs):
        super().__init__(n_clusters, random_state)
        self.kwargs = kwargs
        self.model = KMeans(
            n_clusters=n_clusters,
            random_state=random_state,
            **kwargs
        )

    def fit(self, X: np.ndarray):
        """Fit KMeans on all data at once."""
        self.model.fit(X)
        self._is_fitted = True
        return self

    def fit_iterative(self, data_loader, load_func: Optional[callable] = None):
        """KMeans doesn't support iterative fitting - loads all data first."""
        print("Warning: KMeans doesn't support true iterative fitting. Loading all data...")

        all_data = []
        for item in data_loader:
            data = load_func(item) if load_func else item
            if data is not None and data.size > 0:
                all_data.append(data)

        if all_data:
            X = np.vstack(all_data)
            self.fit(X)
        return self


class MiniBatchKMeansClustering(ClusteringAlgorithm):
    """MiniBatch KMeans with support for iterative fitting."""

    def __init__(
        self,
        n_clusters: int,
        batch_size: int = 1024,
        random_state: int = 0,
        **kwargs
    ):
        super().__init__(n_clusters, random_state)
        self.batch_size = batch_size
        self.kwargs = kwargs
        self.model = MiniBatchKMeans(
            n_clusters=n_clusters,
            batch_size=batch_size,
            random_state=random_state,
            **kwargs
        )

    def fit(self, X: np.ndarray):
        """Fit MiniBatchKMeans on all data at once."""
        self.model.fit(X)
        self._is_fitted = True
        return self

    def fit_iterative(
        self,
        data_loader,
        load_func: Optional[callable] = None,
        accumulate_batch_size: Optional[int] = None
    ):
        """
        Fit iteratively using partial_fit.

        Args:
            data_loader: Iterable of file paths or data chunks
            load_func: Function to load data from paths
            accumulate_batch_size: Size to accumulate before partial_fit
                                   (defaults to self.batch_size)
        """
        if accumulate_batch_size is None:
            accumulate_batch_size = self.batch_size

        for batch in descriptor_batch_generator(data_loader, load_func, accumulate_batch_size):
            self.model.partial_fit(batch)

        self._is_fitted = True
        return self


# ============================================================================
# Encoding Implementations
# ============================================================================

class BagOfVisualWords(EncodingAlgorithm):
    """
    Bag of Visual Words encoding.
    Represents an image as a histogram of visual word occurrences.
    """

    def __init__(self, clustering: ClusteringAlgorithm, normalize: bool = True):
        super().__init__(clustering)
        self.normalize = normalize

    def encode(self, descriptors: np.ndarray) -> np.ndarray:
        """
        Encode descriptors into BoVW histogram.

        Args:
            descriptors: Array of shape (n_descriptors, n_features)

        Returns:
            Histogram of visual words of shape (n_clusters,)
        """
        if descriptors is None or descriptors.size == 0:
            return np.zeros(self.clustering.n_clusters)

        # Assign each descriptor to nearest cluster
        labels = self.clustering.predict(descriptors)

        # Build histogram
        histogram = np.bincount(
            labels,
            minlength=self.clustering.n_clusters
        ).astype(np.float32)


        # L2 normalization
        norm = np.linalg.norm(histogram)
        if norm > 0:
            histogram = histogram / norm

        # Normalize l1
        # if self.normalize and histogram.sum() > 0:
        #     histogram /= histogram.sum()

        return histogram

    def encode_batch(self, descriptors_list: List[np.ndarray]) -> np.ndarray:
        """Encode multiple descriptor sets."""
        return np.array([self.encode(desc) for desc in descriptors_list])


class VLAD(EncodingAlgorithm):
    """
    Vector of Locally Aggregated Descriptors.
    Accumulates residuals between descriptors and cluster centers.
    """

    def __init__(self, clustering: ClusteringAlgorithm, normalize: bool = True):
        super().__init__(clustering)
        self.normalize = normalize

    def encode(self, descriptors: np.ndarray) -> np.ndarray:
        """
        Encode descriptors (of a single image) into VLAD vector.

        Args:
            descriptors: Array of shape (n_descriptors, n_features)

        Returns:
            VLAD vector of shape (n_clusters * n_features,)
        """
        if descriptors is None or descriptors.size == 0:
            n_features = self.clustering.cluster_centers_.shape[1]
            return np.zeros(self.clustering.n_clusters * n_features)

        # Assign descriptors to clusters
        labels = self.clustering.predict(descriptors)
        centers = self.clustering.cluster_centers_

        # Compute VLAD
        n_clusters, n_features = centers.shape
        vlad = np.zeros((n_clusters, n_features))

        for idx in range(n_clusters):
            # Find descriptors assigned to cluster i
            mask = (labels == idx)
            if np.any(mask):
                # Sum of residuals
                residuals = descriptors[mask] - centers[idx]
                vlad[idx] = residuals.sum(axis=0)

        # Flatten and normalize
        vlad = vlad.flatten()

        # Intra-normalization: L2 normalize each cluster's residuals
        vlad = vlad.reshape(n_clusters, n_features)
        vlad = normalize(vlad, norm='l2', axis=1)
        vlad = vlad.flatten()

        # Power normalization (reduces burstiness)
        vlad = np.sign(vlad) * np.sqrt(np.abs(vlad))

        if self.normalize:
            # L2 normalization (per image)
            vlad = normalize(vlad.reshape(1, -1), norm='l2').flatten()

        return vlad

    def encode_batch(self, image_list: List[np.ndarray], load_function) -> np.ndarray:
        """
        Encode a batch of batch of images (descriptors) into VLAD vectors.

        Args:
            descriptors_list: List of each image descriptors,
                            each of shape (n_descriptors, n_features)

        Returns:
            Array of VLAD vectors of shape
            (batch_size, n_clusters * n_features)
        """
        vlad_vectors = []

        for item in image_list:
            descriptors = load_function(item)
            vlad = self.encode(descriptors)
            vlad_vectors.append(vlad)

        return np.vstack(vlad_vectors)



class FisherVector(EncodingAlgorithm):
    """
    Fisher Vector encoding.
    Note: This is a simplified implementation. For production use,
    consider using a GMM-based approach.
    """

    def __init__(self, clustering: ClusteringAlgorithm, normalize: bool = True):
        super().__init__(clustering)
        self.normalize = normalize
        print("Note: This is a simplified Fisher Vector using K-means.")
        print("For full Fisher Vectors, use sklearn.mixture.GaussianMixture")

    def encode(self, descriptors: np.ndarray) -> np.ndarray:
        """
        Simplified Fisher Vector encoding using K-means centers.

        Returns:
            Fisher vector of shape (n_clusters * n_features * 2,)
        """
        if descriptors is None or descriptors.size == 0:
            n_features = self.clustering.cluster_centers_.shape[1]
            return np.zeros(self.clustering.n_clusters * n_features * 2)

        labels = self.clustering.predict(descriptors)
        centers = self.clustering.cluster_centers_

        n_clusters, n_features = centers.shape
        fv = np.zeros((n_clusters, n_features, 2), dtype=np.float32)

        for i in range(n_clusters):
            mask = labels == i
            if np.any(mask):
                cluster_desc = descriptors[mask]
                n_k = cluster_desc.shape[0]

                # Mean deviation
                mean_dev = (cluster_desc - centers[i]).sum(axis=0) / np.sqrt(n_k)
                fv[i, :, 0] = mean_dev

                # Variance deviation (simplified)
                var_dev = ((cluster_desc - centers[i]) ** 2).sum(axis=0) / np.sqrt(n_k)
                fv[i, :, 1] = var_dev - 1.0

        fv = fv.flatten()

        if self.normalize:
            norm = np.linalg.norm(fv)
            if norm > 0:
                fv /= norm

        return fv

    def encode_batch(self, descriptors_list: List[np.ndarray]) -> np.ndarray:
        """Encode multiple descriptor sets."""
        return np.array([self.encode(desc) for desc in descriptors_list])


# ============================================================================
# High-Level Pipeline
# ============================================================================

class VisualEncodingPipeline:
    """
    High-level pipeline combining clustering and encoding.
    """

    def __init__(
        self,
        clustering: ClusteringAlgorithm,
        encoding: EncodingAlgorithm
    ):
        self.clustering = clustering
        self.encoding = encoding

    def fit(self, descriptors: Union[np.ndarray, List], load_func: Optional[callable] = None):
        """
        Fit the clustering model.

        Args:
            descriptors: Either:
                - np.ndarray: All descriptors stacked
                - List: List of file paths or data chunks for iterative fitting
            load_func: Function to load descriptors from paths (for iterative fitting)
        """
        if isinstance(descriptors, np.ndarray):
            self.clustering.fit(descriptors)
        else:
            self.clustering.fit_iterative(descriptors, load_func)
        return self

    def transform(self, descriptors: np.ndarray) -> np.ndarray:
        """Encode a single set of descriptors."""
        return self.encoding.encode(descriptors)

    def fit_transform(self, train_descriptors, transform_descriptors: np.ndarray):
        """Fit on training data and transform other data."""
        self.fit(train_descriptors)
        return self.transform(transform_descriptors)

    def transform_batch(self, descriptors_list: List[np.ndarray]) -> np.ndarray:
        """Encode multiple sets of descriptors."""
        return self.encoding.encode_batch(descriptors_list)


# ============================================================================
# Example Usage
# ============================================================================

if __name__ == "__main__":
    # Example 1: BoVW with MiniBatchKMeans (iterative fitting)
    print("=== Example 1: BoVW with MiniBatch KMeans ===")

    # Create clustering
    clustering = MiniBatchKMeansClustering(n_clusters=64, batch_size=1024)

    # Create encoding
    bovw = BagOfVisualWords(clustering, normalize=True)

    # Create pipeline
    pipeline = VisualEncodingPipeline(clustering, bovw)

    # Simulate file loading
    def load_descriptors(filepath):
        return np.random.rand(100, 128).astype(np.float32)

    train_files = [f"train_{i}.npy" for i in range(50)]

    # Fit iteratively
    pipeline.fit(train_files, load_func=load_descriptors)

    # Encode a single image
    test_desc = np.random.rand(80, 128).astype(np.float32)
    bovw_vector = pipeline.transform(test_desc)
    print(f"BoVW vector shape: {bovw_vector.shape}")
    print(f"BoVW vector sum: {bovw_vector.sum():.3f}")

    # Example 2: VLAD with standard KMeans
    print("\n=== Example 2: VLAD with KMeans ===")

    # All data fits in memory
    all_train_data = np.random.rand(5000, 128).astype(np.float32)

    clustering2 = KMeansClustering(n_clusters=32)
    vlad = VLAD(clustering2, normalize=True)
    pipeline2 = VisualEncodingPipeline(clustering2, vlad)

    pipeline2.fit(all_train_data)
    vlad_vector = pipeline2.transform(test_desc)
    print(f"VLAD vector shape: {vlad_vector.shape}")

    # Example 3: Batch encoding
    print("\n=== Example 3: Batch encoding ===")

    test_images = [np.random.rand(80, 128).astype(np.float32) for _ in range(5)]
    encoded_batch = pipeline.transform_batch(test_images)
    print(f"Batch encoded shape: {encoded_batch.shape}")

    # Example 4: Compare different encodings
    print("\n=== Example 4: Different encodings comparison ===")

    clustering3 = MiniBatchKMeansClustering(n_clusters=64)
    clustering3.fit(all_train_data)

    encodings = {
        "BoVW": BagOfVisualWords(clustering3),
        "VLAD": VLAD(clustering3),
        "Fisher": FisherVector(clustering3)
    }

    for name, enc in encodings.items():
        vector = enc.encode(test_desc)
        print(f"{name:10s} - Shape: {vector.shape}, Norm: {np.linalg.norm(vector):.3f}")

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

k = 256              # number of clusters
# k=3 around 12% (same as normal kmeans)
# k=30 around 43.5% with is actually higher than reduced sift features
# k = 200 around 55%
# k = 2000 around 66% but 22 min training while 200 took 5 min per cv
# k=2000 with batch size 10k, 66% with 23 min traing
# k=5000 10k size, 69%
# Note that with minibatchKmeans you can increase the number of clusters without getting to penalized (in time) by it
# They have better scaling for number of clusters than KMeans
# VLAD -> Faster clustering but longer training 72% precision with k=256 around 1hour
batch_size = 8000        # minibatch size
desc_folder = "descriptors"

skf = StratifiedKFold(n_splits=5, shuffle=True)
accuracies = []

for train_idx, val_idx in skf.split(X_train, y_train):

    # --- 1) Build list of descriptor filenames for this fold ---
    train_files = [f"desc_{i}.npy" for i in train_idx]
    val_files   = [f"desc_{i}.npy" for i in val_idx]

    # --- 2) Train MiniBatchKMeans only on training descriptors ---
    clustering = MiniBatchKMeansClustering(n_clusters=256, batch_size=4096)
    clustering.fit_iterative(train_files, load_desc_safe)

    vlad = VLAD(clustering)
    X_tr_hist = vlad.encode_batch(train_files, load_desc_safe)
    X_val_hist = vlad.encode_batch(val_files, load_desc_safe)

    print("Ready to train")

    # Label
    y_tr = [y_train[i] for i in train_idx]
    y_val = [y_train[i] for i in val_idx]

    # --- 5) Train your classifier ---
    clf = SVC(kernel='linear')
    clf.fit(X_tr_hist, y_tr)

    acc = clf.score(X_val_hist, y_val)
    accuracies.append(acc)
    print("Fold accuracy:", acc)


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.svm import SVC

# For cross-validation we use StratifiedKFolds because it is a variation of KFolds that keeps the proportion of each class across the sets
skf = StratifiedKFold(n_splits=5, shuffle=True)
k = 3
# 30 clusters, 23% accuracy and 17min training
# 30 clusters akaze, 35% accuracy and 51min training
# 30 clusters sift500, 38% accuracy and 22 min training
accuracies = []

for train_idx, val_idx in skf.split(X_train, y_train):
  train_descriptors = [image_descriptors[i] for i in train_idx]
  val_descriptors   = [image_descriptors[i] for i in val_idx]

  # Stack all non-empty descriptors for training KMeans
  useful_descriptors = [d for d in train_descriptors if d is not None]
  useful_descriptors = np.vstack(useful_descriptors)

  # Fit KMeans on train descriptors only
  # After learning the kmeans we use it to label each descriptor
  # Then we will know which cluster will have more descriptors associated
  kmeans = KMeans(n_clusters=k, random_state=0).fit(useful_descriptors)

  def compute_histograms(descriptor_list):
        hists = []
        for desc in descriptor_list:
            if desc is not None:
                # Assign each descriptor to nearest cluster center
                cluster_labels = kmeans.predict(desc)
                # Create histogram for this image (length = number of clusters)
                hist, _ = np.histogram(cluster_labels, bins=np.arange(k + 1))
            else:
                # Image with no descriptors -> zero histogram
                hist = np.zeros(k, dtype=int)
            hists.append(hist)
        return normalize(np.array(hists), norm='l2')

  # BoVW histograms
  X_tr_hist = compute_histograms(train_descriptors)
  X_val_hist = compute_histograms(val_descriptors)

  # Label
  y_tr = [y_train[i] for i in train_idx]
  y_val = [y_train[i] for i in val_idx]

  # Train classifier
  clf = SVC(kernel='linear')
  clf.fit(X_tr_hist, y_tr)

  # Validate
  acc = clf.score(X_val_hist, y_val)
  accuracies.append(acc)

print("Mean CV accuracy:", np.mean(accuracies))
print("Std CV accuracy:", np.std(accuracies))


In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import normalize
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

k = 256              # number of clusters
# k=3 around 12% (same as normal kmeans)
# k=30 around 43.5% with is actually higher than reduced sift features
# k = 200 around 55%
# k = 2000 around 66% but 22 min training while 200 took 5 min per cv
# k=2000 with batch size 10k, 66% with 23 min traing
# k=5000 10k size, 69%
# Note that with minibatchKmeans you can increase the number of clusters without getting to penalized (in time) by it
# They have better scaling for number of clusters than KMeans
batch_size = 8000        # minibatch size
desc_folder = "descriptors"

skf = StratifiedKFold(n_splits=5, shuffle=True)
accuracies = []

for train_idx, val_idx in skf.split(X_train, y_train):

    # --- 1) Build list of descriptor filenames for this fold ---
    train_files = [f"desc_{i}.npy" for i in train_idx]
    val_files   = [f"desc_{i}.npy" for i in val_idx]

    # --- 2) Train MiniBatchKMeans only on training descriptors ---
    kmeans = MiniBatchKMeans(
        n_clusters=k,
        batch_size=batch_size,
        random_state=0
    )

    batch = []
    current_size = 0

    for file in train_files:
        desc = load_desc_safe(file)

        # Ignore empty descriptors
        if desc is None or desc.size == 0:
          continue

        # Append descriptors for streaming
        batch.append(desc)
        current_size += desc.shape[0]

        # If enough descriptors → partial_fit
        if current_size >= batch_size:
            kmeans.partial_fit(np.vstack(batch))
            batch = []
            current_size = 0

    # Fit final batch if any
    if len(batch) > 0:
        kmeans.partial_fit(np.vstack(batch))

    # --- 3) Function to compute histograms safely ---
    def compute_histograms(file_list):
        hists = []
        for file in file_list:
            desc = load_desc_safe(file)
            if desc is None:
                hist = np.zeros(k, dtype=int)
            else:
                labels = kmeans.predict(desc)
                hist, _ = np.histogram(labels, bins=np.arange(k+1))
            hists.append(hist)
        return normalize(np.array(hists), norm='l2')

    # --- 4) Build BoVW histograms ---
    X_tr_hist = compute_histograms(train_files)
    X_val_hist = compute_histograms(val_files)

    # Label
    y_tr = [y_train[i] for i in train_idx]
    y_val = [y_train[i] for i in val_idx]

    # --- 5) Train your classifier ---
    clf = SVC(kernel='linear')
    clf.fit(X_tr_hist, y_tr)

    acc = clf.score(X_val_hist, y_val)
    accuracies.append(acc)
    print("Fold accuracy:", acc)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100,random_state=0)
clf.fit(X_tr_hist, y_tr)

acc = clf.score(X_val_hist, y_val)
print("Fold accuracy:", acc)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=350, random_state=42)
classifier.fit(X_tr_hist, y_tr)
acc = classifier.score(X_val_hist, y_val)
print("Fold accuracy:", acc)

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_clf = GaussianNB()
nb_clf.fit(X_tr_hist, y_tr)
y_pred_nb = nb_clf.predict(X_val_hist)
acc_nb = nb_clf.score(X_val_hist, y_val)
print(f"Accuracy Naïve Bayes: {acc_nb}")

In [ ]:
from sklearn.linear_model import LogisticRegression

softmax_clf = LogisticRegression(
    multi_class='multinomial',  # for multiclass classification (Softmax)
    solver='lbfgs',
    max_iter=500
)
softmax_clf.fit(X_tr_hist, y_tr)
y_pred_softmax = softmax_clf.predict(X_val_hist)
acc_softmax = softmax_clf.score(X_val_hist, y_val)
print(f"Accuracy Softmax Regression: {acc_softmax}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(
    max_depth=15,
    random_state=42
)

dt_clf.fit(X_tr_hist, y_tr)
y_pred_dt = dt_clf.predict(X_val_hist)
acc_dt = dt_clf.score(X_val_hist, y_val)
print(f"Accuracy Decision Tree: {acc_dt}")